# Inserción de las temperatuuras


### Importaciones necesarias

In [1]:
import pandas as pd
import numpy as np
import pymysql
from pymysql.constants import CLIENT
from dotenv import load_dotenv
import os

### Cargar las variables de entorno desde el archivo .env


In [2]:
load_dotenv()

# Obtener los parámetros de conexión
DB_HOST = os.getenv('DB_HOST')
DB_USER = os.getenv('DB_USER')
DB_PASSWORD = os.getenv('DB_PASSWORD')
DB_NAME = os.getenv('DB_NAME')

### Conexión a la base de datos

In [3]:

conexion = pymysql.connect(
    host=DB_HOST,
    user=DB_USER,
    password=DB_PASSWORD,
    database=DB_NAME,
    client_flag=CLIENT.MULTI_STATEMENTS
)
cursor = conexion.cursor()

## Carga y Exploración Inicial

En esta sección se carga el dataset y se realiza un análisis exploratorio preliminar para conocer su estructura, dimensiones y calidad. Se muestran las primeras filas, se obtienen estadísticas descriptivas y se revisa la presencia de valores nulos o duplicados.

In [4]:
ruta = '../../data/fuentes/climaticos/Environment_Temperature_change_E_All_Data_NOFLAG.csv'
df_original = pd.read_csv(ruta, encoding='latin-1', sep=',')
print('Cabezera del DataFrame original: ')
print(df_original.head())
print('Dimensiones del DataFrame original: ')
print(df_original.info())

Cabezera del DataFrame original: 
   Area Code         Area  Months Code    Months  Element Code  \
0          2  Afghanistan         7001   January          7271   
1          2  Afghanistan         7001   January          6078   
2          2  Afghanistan         7002  February          7271   
3          2  Afghanistan         7002  February          6078   
4          2  Afghanistan         7003     March          7271   

              Element Unit  Y1961  Y1962  Y1963  ...  Y2010  Y2011  Y2012  \
0  Temperature change   °C  0.777  0.062  2.744  ...  3.601  1.179 -0.583   
1  Standard Deviation   °C  1.950  1.950  1.950  ...  1.950  1.950  1.950   
2  Temperature change   °C -1.743  2.465  3.919  ...  1.212  0.321 -3.201   
3  Standard Deviation   °C  2.597  2.597  2.597  ...  2.597  2.597  2.597   
4  Temperature change   °C  0.516  1.336  0.403  ...  3.390  0.748 -0.527   

   Y2013  Y2014  Y2015  Y2016  Y2017  Y2018  Y2019  
0  1.233  1.755  1.943  3.416  1.201  1.996  2.951  


### Valores Nulos del Dataset

In [5]:
print(df_original.isnull().sum().to_string())


Area Code          0
Area               0
Months Code        0
Months             0
Element Code       0
Element            0
Unit               0
Y1961           1369
Y1962           1334
Y1963           1362
Y1964           1404
Y1965           1375
Y1966           1292
Y1967           1309
Y1968           1311
Y1969           1330
Y1970           1348
Y1971           1353
Y1972           1333
Y1973           1262
Y1974           1282
Y1975           1376
Y1976           1447
Y1977           1399
Y1978           1329
Y1979           1366
Y1980           1373
Y1981           1380
Y1982           1419
Y1983           1451
Y1984           1397
Y1985           1440
Y1986           1388
Y1987           1372
Y1988           1383
Y1989           1399
Y1990           1417
Y1991           1498
Y1992           1302
Y1993           1341
Y1994           1283
Y1995           1247
Y1996           1217
Y1997           1347
Y1998           1286
Y1999           1332
Y2000           1314
Y2001        

In [6]:
nulls_by_country_total = df_original.groupby('Area').apply(lambda group: group.isnull().sum().sum())
nulls_with_data = nulls_by_country_total[nulls_by_country_total > 0]
print(nulls_with_data.to_string())


Area
American Samoa                                   184
Antigua and Barbuda                               92
Armenia                                         1530
Aruba                                            158
Azerbaijan                                      1530
Barbados                                         122
Belarus                                         1530
Belgium                                         1666
Belgium-Luxembourg                               680
Bosnia and Herzegovina                          1530
Burundi                                          752
Cabo Verde                                       150
Cayman Islands                                   266
Central African Republic                           8
Central Asia                                    1530
Christmas Island                                1319
Cocos (Keeling) Islands                            2
Comoros                                           50
Congo                                    

C:\Users\jcall\AppData\Local\Temp\ipykernel_19664\417736343.py:1: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  nulls_by_country_total = df_original.groupby('Area').apply(lambda group: group.isnull().sum().sum())


## Perfilado y Limpieza de Datos

Se realiza un perfilado detallado para identificar y solucionar problemas de calidad, como valores nulos o duplicados. Posteriormente, se eliminan las filas problemáticas para asegurar que el dataset esté limpio y listo para su transformación.

En este caso, realizamos las transformaciones necesarias para poder insertar correctamente los datos en los paises que ya están almacenados en la base de datos 

In [7]:
# ————————————————————————————————————————————————
# Celda 1 (unificada): normalizar, excepciones, explode y mapear pais_id
# ————————————————————————————————————————————————

# 1) Normalizar 'Area' a minúsculas y sin espacios
df_original['area_norm'] = (
    df_original['Area']
      .astype(str)
      .str.strip()
      .str.lower()
)

# 2) Cargar dimensión Paises: (codigo, nombre_en) → dict nombre_en_normalizado → codigo
cursor.execute("SELECT codigo, nombre_en FROM Paises;")
dim_paises = {
    nombre_en.strip().lower(): codigo
    for codigo, nombre_en in cursor.fetchall()
}

print(f"⚠️ Se cargaron {len(dim_paises)} países de la dimensión Paises")

# 3) Diccionario de excepciones: area_norm → nombre_en(s) exacto(s) en BD
exceptions = {
    'belgium-luxembourg':'belgium',
    'british virgin islands':'virgin islands (british)',
    'channel islands':None,
    'china, hong kong sar':'hong kong',
    'china, macao sar':'macao',
    'china, mainland': None,
    'china, taiwan province of':'taiwan (province of china)',
    "democratic people's republic of korea":"korea (the democratic people's republic of)",
    'democratic republic of the congo':'congo (the democratic republic of the)',
    'ethiopia pdr':'ethiopia',
    'falkland islands (malvinas)':'falkland islands (the) [malvinas]',
    'french southern and antarctic territories':'french southern territories',
    'midway island':None,
    'namibia':None,
    'netherlands antilles (former)':  None,
    'pacific islands trust territory':None,
    'pitcairn islands':'pitcairn',
    'republic of korea':'korea (the republic of)',
    'republic of moldova':'moldova (the republic of)',
    'serbia and montenegro':['serbia','montenegro'],
    'sudan (former)':['sudan','south sudan'],
    'svalbard and jan mayen islands': 'svalbard and jan mayen',
    'turkey':'türkiye',
    'united republic of tanzania':'tanzania, the united republic of',
    'united states virgin islands':'virgin islands (u.s.)',
    'ussr':None,
    'wake island':None,
    'wallis and futuna islands':'wallis and futuna',
}

# 4) Mapear area_norm → area_db_name (str, list o NaN)
df_original['area_db_name'] = df_original['area_norm'].map(
    lambda x: exceptions[x] if x in exceptions else x
)

# 5) Asegurar listas y 'explode' para casos múltiples
df_original['area_db_list'] = df_original['area_db_name'].apply(
    lambda v: [] if v is None else (v if isinstance(v, list) else [v])
)
df_original = df_original.explode('area_db_list')

# 6) Mapear area_db_list (nombre_en normalizado) → codigo
df_original['pais_id'] = (
    df_original['area_db_list']
      .str.strip()
      .str.lower()
      .map(dim_paises)
)

# 7) Filtrar fuera filas sin mapeo válido
antes = len(df_original)
df_original = df_original[df_original['pais_id'].notna()]
print(f"⚠️ Se descartaron {antes - len(df_original)} filas sin país válido")


⚠️ Se cargaron 248 países de la dimensión Paises
⚠️ Se descartaron 1734 filas sin país válido


En este paso se aplican transformaciones para adaptar los datos a la estructura requerida. Por ejemplo, se transforma el campo `Months Code` en un identificador de período que servirá para relacionar los datos con la tabla de dimensiones `Periodos` en la base de datos. 

In [8]:
# 1) Definir las columnas de años
year_columns = [c for c in df_original.columns if c.startswith('Y')]

# 2) Filtro anual (Months Code = 7020)
anual_filter = df_original['Months Code'] == 7020

# 3) Filtro por tipo de parámetro (variación de temperatura)
cambio_filter = df_original['Element'] == 'Temperature change'

# 4) Función para pivotar (melt) e incluir `pais_id`
def crear_dataset(filtro_periodo, filtro_param, df):
    df_temp = df[filtro_periodo & filtro_param].copy()
    df_temp = pd.melt(
        df_temp,
        id_vars=['pais_id', 'Area Code', 'area_db_list', 'Months Code', 'Unit'],
        value_vars=year_columns,
        var_name='Year',
        value_name='Value'
    )
    df_temp['Year'] = df_temp['Year'].str.lstrip('Y').astype(int)
    return df_temp

# 5) Generar SOLO el dataset anual de cambio de temperatura
cambio_temperatura_anual = crear_dataset(anual_filter, cambio_filter, df_original)

# 6) Verificación
print(f"cambio_temperatura_anual: {cambio_temperatura_anual.shape[0]} filas, "
      f"pais_id nulos = {cambio_temperatura_anual['pais_id'].isnull().sum()}")


cambio_temperatura_anual: 13865 filas, pais_id nulos = 0


Los datos se dividen en subconjuntos (datasets) según el tipo de parámetro y el período:
- **Derivación Estándar:** Datos para desviación estándar (mensual, trimestral, anual).
- **Cambio de Temperatura:** Datos de cambio de temperatura (mensual, trimestral, anual).

In [9]:
datasets = {
    "cambio_temperatura_anual": cambio_temperatura_anual,
}

for name, df in datasets.items():
    print(f"Valores nulos para {name}:")
    null_counts = df.isnull().sum()
    print(null_counts[null_counts > 0].to_string() if (null_counts > 0).any() else "Sin nulos")
    print("\n" + "-"*40 + "\n")


Valores nulos para cambio_temperatura_anual:
Value    1632

----------------------------------------



In [10]:
# Eliminar nulos y duplicados únicamente del dataset anual
for name, df in datasets.items():
    filas_originales = df.shape[0]
    df_clean = df.dropna()
    df_clean = df_clean.drop_duplicates()
    filas_limpias = df_clean.shape[0]
    print(f"{name}: {filas_originales} filas originales -> {filas_limpias} filas después de eliminar nulos")
    datasets[name] = df_clean

# Reasignar variable
cambio_temperatura_anual = datasets["cambio_temperatura_anual"]


cambio_temperatura_anual: 13865 filas originales -> 12233 filas después de eliminar nulos


## Inserción de los datos

En esta sección se realiza la inserción de los datos limpios y transformados en una base de datos MySQL. Se establecen conexiones utilizando `pymysql`, y se insertan los datos en las tablas respectivas



In [11]:
# Obtener el id del indicador (asegura que exista)
cursor.execute("SELECT id FROM Indicadores WHERE codigo = %s LIMIT 1;", ("temperaturas",))
row = cursor.fetchone()
if not row:
    raise RuntimeError("No existe un indicador con codigo='temperaturas'")
indic_temp_id = row[0]

# Preparar datos (solo anual)
fact_hechos = []
for _, row in cambio_temperatura_anual.iterrows():
    pid  = str(row['pais_id'])
    anio = int(row['Year'])
    val  = float(row['Value'])
    fact_hechos.append((pid, anio, indic_temp_id, val))

print(f"Preparadas {len(fact_hechos)} filas para insertar en Hechos.")

# INSERT con 4 placeholders
sql_hechos = """
INSERT INTO Hechos 
  (pais_id, anio, indicador_id, valor)
VALUES (%s, %s, %s, %s);
"""
cursor.executemany(sql_hechos, fact_hechos)
conexion.commit()

print(f"Cambios de temperatura anuales insertados correctamente en Hechos ({len(fact_hechos)} filas).")


Preparadas 12233 filas para insertar en Hechos.
Cambios de temperatura anuales insertados correctamente en Hechos (12233 filas).
